# AMSC698K Homework 6
##### Elijah Kin & Noorain Noorani

In [1]:
import math
import numpy as np
import qiskit
from qiskit import QuantumCircuit
from qiskit_aer import Aer
from qiskit.visualization import plot_histogram
qiskit.__version__

'1.2.4'

In [2]:
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister
from qiskit_aer import Aer

### 1a - (60pts) Create a program for Shor's algorithm, i.e. for factorizing an integer N, including a quantum circuit to find the order r of a modulo $N (a^r := 1 mod N)$. 

First, create functions that perform the required processing steps (you may adapt the corresponding functions presented in class):

1) Create helper functions:

    (a) to check whether integer N is a perfect power ($N=p^q)$;

    (b) to check whether N and the base a (with $1<a<N$) are coprime;

    (c) to extract the exponent r and the factors of N.

In [3]:
def is_perfect_power(n):
    """
    Check if integer n is a perfect power (n = p^q with q > 1).
    
    Returns:
        (True, p, q) if n is a perfect power,
        (False, None, None) otherwise.
    """
    # Loop over possible exponents q starting at 2.
    # The upper limit for q is taken from log2(n) since 2 is the smallest possible base.
    max_exponent = int(math.log(n, 2)) + 1
    for q in range(2, max_exponent + 1):
        p = int(round(n ** (1.0 / q)))
        # Check if p^q exactly equals n
        if p ** q == n:
            return True, p, q
    return False, None, None

In [4]:
def are_coprime(n, a):
    """
    Check if integer n and base a (with 1 < a < n) are coprime.
    
    Returns:
        True if gcd(n, a) == 1, False otherwise.
    """
    if not (1 < a < n):
        raise ValueError("Base a must satisfy 1 < a < n")
    return math.gcd(n, a) == 1

In [5]:
def order_and_factors(n, a):
    """
    Compute the order r of a modulo n (i.e., the smallest positive integer r 
    such that a^r ≡ 1 mod n). Then, if r is even, attempt to extract non-trivial 
    factors of n by computing:
        factor1 = gcd(a^(r/2) - 1, n)
        factor2 = gcd(a^(r/2) + 1, n)
    
    Returns:
        r: the order of a modulo n,
        factors: a tuple (factor1, factor2) if r is even, otherwise None.
    """
    # Find the order r (smallest r such that a^r mod n == 1)
    r = None
    for i in range(1, n):
        if pow(a, i, n) == 1:
            r = i
            break

    if r is None:
        raise ValueError("No order found; check inputs for coprimality.")
    
    factors = None
    # Only proceed with factor extraction if r is even.
    if r % 2 == 0:
        # Compute a^(r/2) mod n
        x = pow(a, r // 2, n)
        # Calculate possible factors using the gcd method.
        factor1 = math.gcd(x - 1, n)
        factor2 = math.gcd(x + 1, n)
        factors = (factor1, factor2)
    
    return r, factors

In [6]:
N = 16  # Try a perfect power: 16 = 2^4
print("Checking if N =", N, "is a perfect power:")
is_pp, base, exponent = is_perfect_power(N)
if is_pp:
    print(f"  Yes, {N} = {base}^{exponent}")
else:
    print("  No, it is not a perfect power.")

# Example for coprimality check:
n = 15
a = 7
print(f"\nChecking if {n} and {a} are coprime:")
print("  Coprime?" , are_coprime(n, a))

# Example for order and factor extraction:
# Here we use n = 15 and a = 7. Note that 7 and 15 are coprime.
print(f"\nExtracting order and factors for n = {n} with base a = {a}:")
r, factors = order_and_factors(n, a)
print("  Order r =", r)
if factors:
    print("  Extracted factors:", factors)
else:
    print("  Order is not even; no factors extracted via this method.")

Checking if N = 16 is a perfect power:
  Yes, 16 = 4^2

Checking if 15 and 7 are coprime:
  Coprime? True

Extracting order and factors for n = 15 with base a = 7:
  Order r = 4
  Extracted factors: (3, 5)


In [7]:
## check if N is perfect power, ie. N=m^b (b=2,3,..)
def check_if_perfect_power(N):
    b = 2
    while 2**b <= N:
        a = 1 ; c = N
        while c - a >= 2:
            m = (a + c) // 2
            p = m**b if m**b <= N else N+1
            if p == N:
                return (m,b)
            if p < N:
                a = m
            else:
                c = m
        b += 1
    return (0,0)

## calculate gcd and modular inverse using Euclid's algorithm
## recursive function 'xgcd' updates a,b,x,y
def xgcd(a, b, x=0, y=1):
    # returns (gcd, x, y)
    if a == 0:
        return (b, 0, 1)
    g, x, y = xgcd(b % a, a, x, y)
    return (g, y - (b // a) * x, x)

## dummy function 'egcd' to return only 'gcd'
def egcd(a, b):
    return xgcd(a, b)[0]

def modinv(a, m):
    g, x, y = xgcd(a, m)
    if g != 1:
        raise Exception("modular inverse does not exist")
    else:
        # add m to account for negative x
        return (x % m + m) % m

## find coprime a to N from 'list_for_a' or range(2,N)
def get_coprime(N, list_for_a=[]):
    # default list starting with a=2
    if len(list_for_a) == 0:
        alist = list(range(2,N))
    else:
        alist = [ i for i in list_for_a if i>1 and i<N ]
        print(alist)
    for a in alist:
        if egcd(a, N) == 1:
            return a
    return 2

In [8]:
## extract factors of N based on results in counts_dict
def get_factors(counts_dict, a, N, qubits=[" ",0], nshots=0, mincounts=2):
    # qubits=list of qubits which the state is encoded in 
    # or: qubits=("separator",item_number) if state is encoded in 
    #                      bitstring[item_number] separated by "separator"
    # or: qubits=QuantumRegister containing the state
    if isinstance(qubits, (list,tuple)):
        myflag = 0 if len(qubits)==2 and isinstance(qubits[0], str) else 1
    elif isinstance(qubits, int) and qubits > 0:
        qubits = list(range(qubits))
        myflag = 1
    elif isinstance(qubits, QuantumRegister):
        qubits = list(range(qubits.size))
        myflag = 1

    if isinstance(counts_dict, (np.ndarray, list)):
        if isinstance(counts_dict, np.ndarray):
            counts_dict = counts_dict.tolist()
        counts_dict = dict.fromkeys(counts_dict, 1)
    if nshots == 0:
        nshots = np.sum(list(counts_dict.values()))
        
    outdict = {}  # {"state": [x, gcd(a^(x/2)-1,N), gcd(a^(x/2)+1,N), cnts]}
    for key,cnts in counts_dict.items():
        if cnts < mincounts:
            continue
        if myflag == 0:
            state = key.split(qubits[0])[qubits[1]]
        else:
            state = "".join([ k for j,k in enumerate(list(key)) if j in qubits ])
        xval = int(state,2)
        # remove all odd and trivial solutions
        if xval == 0 or xval == N or xval % 2 == 1:
            continue
        if xval > 255:
            continue   # potential overflow 
        x = int(a ** (xval/2))
        if (x+1) % N == 0: 
            continue
        # a^(r/2)+1 or a^(r/2)-1 should share factors with N
        factors = egcd(x-1, N), egcd(x+1, N) 
        if factors[0] in [1,N] or factors[1] in [1,N]:
            continue
        outdict.update({ state : [xval, factors[0], factors[1], cnts/nshots] })
    return outdict

2) Create quantum circuits:

- to perform QFT and inverse QFT on N qubits;
- to perform the double-controlled modular addition of $a$ in the Fourier space and its inverse;
- to perform controlled modular multiplication by $a$ 

In [9]:
## functions to create the quantum circuit

from qiskit import QuantumCircuit, QuantumRegister

## create the QFT 
def QFT(qcirc, qreg, nqubits, do_swaps=False, min_angle=0):    
    for i in reversed(range(nqubits)):
        qcirc.h(qreg[i])         
        for j in reversed(range(i)): 
            # apply CPhase gates only if |angle|>min_angle
            myphase = math.pi/pow(2,(i-j))
            if myphase > min_angle:
                qcirc.cp( myphase, qreg[i], qreg[j] ) 
    if do_swaps:
        for i in range(nqubits//2):
            qcirc.swap(qreg[i], qreg[nqubits-1-i])
    return qcirc

## create the inverse QFT 
def invQFT(qcirc, qreg, nqubits, do_swaps=False, min_angle=0):
    if do_swaps:
        for i in range(nqubits//2):
            qcirc.swap(qreg[i], qreg[nqubits-1-i])    
    for i in range(nqubits): 
        qcirc.h(qreg[i])
        for j in range(i):
            myphase = math.pi/pow(2,(j-i))
            if myphase > min_angle:
                qcirc.cp( -myphase, qreg[j] , qreg[i] )
    return qcirc

In [10]:
## calculate phase shift in the sequential QFT based on a,
## using binary representation of a: if bit=1 add a shift to the angle
def getPhase(a, N):
    bitstr = bin(int(a))[2:].zfill(N) 
    phase = 0
    for i in range(0, N):
        if bitstr[N-1-i] == '1': 
            phase += math.pow(2, -(N-i))
    phase *= np.pi
    return phase

## calculate phases used for addition in the Fourier Space 
##    (set inverse=True for subtraction)
def getPhases(a, N, inverse=False):
    bitstr = bin(int(a))[2:].zfill(N) 
    phases = np.zeros(N)
    for i in range(N):
        for j in range(i,N):
            if bitstr[j] == '1':
                phases[N-i-1] += math.pow(2, -(j-i))
    phases *= np.pi
    if inverse:
        phases *= -1
    return phases

In [11]:
## perform addition by a in Fourier space
## (or subtraction when setting inverse=True)
def phiADD(qcirc, qreg, a, n, inverse=False):
    myphases = getPhases(a, n, inverse)
    for i in range(n):
        qcirc.p(myphases[i], qreg[i])

## single controlled version of phiADD 
def cphiADD(qcirc, qreg, qctl, a, n, inverse=False):
    myphases = getPhases(a, n, inverse)
    for i in range(n):
        qcirc.cp(myphases[i], qctl, qreg[i])
        
## double controlled version of phiADD (decomposed CCP gate (missing in qiskit))
def ccphiADD(qcirc, qreg ,qctl1, qctl2, a, n, inverse=False):
    myphases = getPhases(a, n, inverse)
    for i in range(n):
        qcirc.cp(myphases[i]/2, qctl1, qreg[i])
        qcirc.cx(qctl2, qctl1)
        qcirc.cp(-myphases[i]/2, qctl1, qreg[i])
        qcirc.cx(qctl2, qctl1)
        qcirc.cp(myphases[i]/2, qctl2, qreg[i])
        
## double controlled modular addition by a
def ccphiADDmodN(qcirc, qreg, qctl1, qctl2, qr_scr, a, N, n):
    ccphiADD(qcirc, qreg, qctl1, qctl2, a, n)
    phiADD(qcirc, qreg, N, n, inverse=True)
    invQFT(qcirc, qreg, n)
    qcirc.cx(qreg[n-1], qr_scr)
    QFT(qcirc, qreg, n)
    cphiADD(qcirc, qreg, qr_scr, N, n)    
    ccphiADD(qcirc, qreg, qctl1, qctl2, a, n, inverse=True)
    invQFT(qcirc, qreg, n)
    qcirc.x(qreg[n-1])
    qcirc.cx(qreg[n-1], qr_scr)
    qcirc.x(qreg[n-1])
    QFT(qcirc, qreg, n)
    ccphiADD(qcirc, qreg, qctl1, qctl2, a, n)

## inverse circuit of double controlled modular addition by a
def ccphiADDmodN_inv(qcirc, qreg, qctl1, qctl2, qr_scr, a, N, n):
    ccphiADD(qcirc, qreg, qctl1, qctl2, a, n, inverse=True)
    invQFT(qcirc, qreg, n)
    qcirc.x(qreg[n-1])
    qcirc.cx(qreg[n-1], qr_scr)
    qcirc.x(qreg[n-1])
    QFT(qcirc, qreg, n)
    ccphiADD(qcirc, qreg, qctl1, qctl2, a, n)
    cphiADD(qcirc, qreg, qr_scr, N, n, inverse=True)
    invQFT(qcirc, qreg, n)
    qcirc.cx(qreg[n-1], qr_scr)
    QFT(qcirc, qreg, n)
    phiADD(qcirc, qreg, N, n)
    ccphiADD(qcirc, qreg, qctl1, qctl2, a, n, inverse=True)

## controlled modular multiplication by a
def cMULTmodN(qcirc, qctl, qreg, qr_scr, a, N, n):
    QFT(qcirc, qr_scr, n+1)
    a_powers = np.mod(a*np.power(2,np.arange(n)),N)
    for i in range(n):
        ccphiADDmodN(qcirc, qr_scr, qreg[i], qctl, qr_scr[n+1], a_powers[i], N, n+1)
    invQFT(qcirc, qr_scr, n+1)
    for i in range(n):
        qcirc.cswap(qctl, qreg[i], qr_scr[i])    
    QFT(qcirc, qr_scr, n+1)
    a_inv = modinv(a, N)
    a_powers = np.mod(a_inv*np.power(2,np.arange(n)),N)
    for i in reversed(range(n)):
        ccphiADDmodN_inv(qcirc, qr_scr, qreg[i], qctl, qr_scr[n+1], a_powers[i], N, n+1)
    invQFT(qcirc, qr_scr, n+1)

3) Create a program that performs the factorization:

- get a positive odd integer N (user input) and check whether N is a perfect power (function 1a);
- get base a and check whether a and N are coprime (function 1b);
with n as the number of bits in N, create 3 quantum registers: a n qubit register (initialized to 1) and a n+2 auxiliary qubit register for performing modular multiplication, and a 2n qubit register to perform the inverse QFT (initialized to uniform superposition);
- apply the modular multiplication gate of power 2k controlled by qubit k (for each k=0,2n) in the 2n qubit register;
- apply the inverse QFT on the 2n qubit register and read it out (measure all 2n qubits);
- extract the exponent r and the factors of N from the measured data (function 1c)

In [12]:
## Shor's algorithm

from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit.circuit import Parameter, IfElseOp
from qiskit_aer.library import save_statevector
from qiskit_aer import Aer
import math, time

backend = Aer.get_backend("aer_simulator")

## N = integer to factorize
## list_for_a = list of numbers to check for coprime of N
##    if the list is empty, check all numbers from 2 to N-1 
## Nshots = number of repetitions (shots) when running the circuit
## mincounts = minimum frequency counts to consider this entry when extracting factors
## version=1: run standard code on 4*nq+2 qubits; 
## version=2: run code with sequential inverse QFT on 2*nq+3 qubits

def test_shor(N, list_for_a=[], Nshots=1000, mincounts=2, version=2):

    ## N=0,1, or even are too simple
    if N < 3 or N % 2 == 0:
        print("Please input an odd number > 1")
        return {}
    
    ## check if N=m^b  
    m,b = check_if_perfect_power(N)
    if b > 0:
        print(f"N is {m}^{b}")
        return {}

    ## get integer g that is coprime with N
    a = get_coprime(N, list_for_a)

    nq = math.ceil(math.log(N,2))         
    tstart = time.time()


    if version == 1:
        
        print(f"Number to factorize N={N}; coprime a={a}; total number of qubits: {4*nq+2} (?)\n")

        ## create quantum and classical registers
        """ Create quantum and classical registers """

        """auxilliary quantum register used in addition and multiplication"""
        qr_scratch = QuantumRegister(nq+2) #aux
        """quantum register where the sequential QFT is performed"""
        qr_qft = QuantumRegister(2*nq) #up_reg
        """quantum register where the multiplications are made"""
        qr_mul = QuantumRegister(nq) #down_reg
        """classical register where the measured values of the QFT are stored"""
        cr_qft = ClassicalRegister(2*nq) #up_classic

        """ Create Quantum Circuit """
        qcirc = QuantumCircuit(qr_mul , qr_qft , qr_scratch, cr_qft)

        print("Create quantum and classical registers")
        
        ## initialize qr_mul to |0...001> and equal superposition in qr_qft
        """ Initialize down register to 1 and create maximal superposition in top register """
        qcirc.h(qr_qft)
        qcirc.x(qr_mul[0])
        print("initialize qr_mul to |0...001> and equal superposition in qr_qft")
        
        ## apply multiplication gates U(a^2^i) to create the exponentiation
        for i in range(0, 2*nq):
            cMULTmodN(qcirc, qr_qft[i], qr_mul, qr_scratch, int(pow(a, pow(2, i))), N, nq)

        print("Apply multiplication gates U(a^2^i) to create the exponentiation")
        
        ## apply inverse QFT
        """ Apply inverse QFT """
        invQFT(qcirc, qr_qft, 2*nq ,do_swaps=True)
        
        ## measure the QFT register
        qcirc.measure(qr_qft,cr_qft)
        
        ## extract x-values from cr_qft
        xvalue_code = (" ",0)

    elif version == 2:
        
        print(f"Number to factorize N={N}; coprime a={a}; total number of qubits: {2*nq+3}\n")

        ## create quantum and classical registers
        qr_qft = QuantumRegister(1)       # single qubit for sequential QFT 
        qr_mul = QuantumRegister(nq)      # where the multiplications are performed
        qr_scr = QuantumRegister(nq+2)    # used for addition and multiplication
        cr_qft = ClassicalRegister(2*nq)  # measured values of the sequential QFT
        cr_scr = ClassicalRegister(1)     # to reset the state of top qubit to 0
                                          #   if previous measurement was 1
        qcirc = QuantumCircuit(qr_mul, qr_qft, qr_scr, cr_qft, cr_scr)

        ## initialize qr_mul to |0...001>
        qcirc.x(qr_mul[0])

        ## auxiliary circuits for conditional gates on cr_scr and cr_qft
        qc_cond_scr, qc_cond_qft = QuantumCircuit(1), QuantumCircuit(1)
        qc_cond_scr.x(0)

        ## create a sequential inverse QFT, apply gates depending on 
        ##   measurements of previous iteration

        for i in range(2*nq):
            
            # reset the top qubit to 0 if previous measurement was 1
            qcirc.compose(IfElseOp((cr_scr,1), qc_cond_scr), qr_qft, inplace=True)

            # apply U(a^2^j) operators
            qcirc.h(qr_qft)
            a_power = a**(2**(2*nq-1-i))
            cMULTmodN(qcirc, qr_qft[0], qr_mul, qr_scr, a_power, N, nq)

            # cycle through all possible values of the classical register 
            #    and apply conditional phase shift
            for j in range(2**i):
                qc_cond_qft.clear()               # reuse auxiliary circuit
                qc_cond_qft.p(getPhase(j,i), 0)
                qcirc.compose(IfElseOp((cr_qft,j), qc_cond_qft), qr_qft, inplace=True)

            qcirc.h(qr_qft)
            qcirc.measure(qr_qft[0], cr_qft[i])
            qcirc.measure(qr_qft[0], cr_scr[0])

        ## skip cr_scr when extracting x-values
        xvalue_code = (" ",1)
    
    ## end elif(version == 2) 

    tsetup = time.time()
    print(f"Quantum circuit with {qcirc.size()} gates generated in {tsetup-tstart:.3f} sec")
    
    # run circuit on some backend and extract factors from sampled states
    counts = backend.run(qcirc, shots=Nshots).result().get_counts()
    print(f"Quantum circuit executed {Nshots} times in {time.time()-tsetup:.3f} sec (got {len(counts)} different values)")

    return get_factors(counts, a, N, qubits=xvalue_code, nshots=Nshots, mincounts=mincounts)

### 1b - (40pts) Test your program on N=15,35,55, then submit the circuit first to "ionq_simulator" without and with noise model, check the results and then run it on an IonQ QPU.

In [24]:
results = test_shor(N=15, list_for_a=[4], Nshots=1000, mincounts=2, version=1)
sum_probs = 0
for state,(x,p,q,prob) in results.items():     
    print(f"Result x={x} ({prob*100}%): (a^(x/2)-1, a^(x/2)+1): {x-1}, {x+1}     => factors:{p},{q}")
    sum_probs += prob*100
print(f"Found factors in {sum_probs:.1f} % of trials")

[4]
Number to factorize N=15; coprime a=4; total number of qubits: 18 (?)

Create quantum and classical registers
initialize qr_mul to |0...001> and equal superposition in qr_qft
Apply multiplication gates U(a^2^i) to create the exponentiation
Quantum circuit with 10105 gates generated in 0.044 sec
Quantum circuit executed 1000 times in 1.481 sec (got 253 different values)
Result x=142 (0.2%): (a^(x/2)-1, a^(x/2)+1): 141, 143     => factors:3,5
Result x=62 (0.3%): (a^(x/2)-1, a^(x/2)+1): 61, 63     => factors:3,5
Result x=10 (0.2%): (a^(x/2)-1, a^(x/2)+1): 9, 11     => factors:3,5
Result x=70 (0.2%): (a^(x/2)-1, a^(x/2)+1): 69, 71     => factors:3,5
Result x=174 (0.2%): (a^(x/2)-1, a^(x/2)+1): 173, 175     => factors:3,5
Result x=134 (0.4%): (a^(x/2)-1, a^(x/2)+1): 133, 135     => factors:3,5
Result x=118 (0.3%): (a^(x/2)-1, a^(x/2)+1): 117, 119     => factors:3,5
Result x=226 (0.5%): (a^(x/2)-1, a^(x/2)+1): 225, 227     => factors:3,5
Result x=6 (0.6%): (a^(x/2)-1, a^(x/2)+1): 5, 7   

In [26]:
results = test_shor(N=21, list_for_a=[5], Nshots=1000, mincounts=2, version=1)
sum_probs = 0
for state,(x,p,q,prob) in results.items():     
    print(f"Result x={x} ({prob*100}%): (a^(x/2)-1, a^(x/2)+1): {x-1}, {x+1}     => factors:{p},{q}")
    sum_probs += prob*100
print(f"Found factors in {sum_probs:.1f} % of trials")

[5]
Number to factorize N=21; coprime a=5; total number of qubits: 22 (?)

Create quantum and classical registers
initialize qr_mul to |0...001> and equal superposition in qr_qft
Apply multiplication gates U(a^2^i) to create the exponentiation
Quantum circuit with 19971 gates generated in 0.072 sec
Quantum circuit executed 1000 times in 41.421 sec (got 635 different values)
Result x=188 (0.3%): (a^(x/2)-1, a^(x/2)+1): 187, 189     => factors:7,3
Result x=68 (0.4%): (a^(x/2)-1, a^(x/2)+1): 67, 69     => factors:3,7
Result x=158 (0.3%): (a^(x/2)-1, a^(x/2)+1): 157, 159     => factors:3,7
Found factors in 1.0 % of trials


In [28]:
results = test_shor(N=35, list_for_a=[13], Nshots=1000, mincounts=2, version=1)
sum_probs = 0
for state,(x,p,q,prob) in results.items():     
    print(f"Result x={x} ({prob*100}%): (a^(x/2)-1, a^(x/2)+1): {x-1}, {x+1}     => factors:{p},{q}")
    sum_probs += prob*100
print(f"Found factors in {sum_probs:.1f} % of trials")

[13]
Number to factorize N=35; coprime a=13; total number of qubits: 26 (?)

Create quantum and classical registers
initialize qr_mul to |0...001> and equal superposition in qr_qft
Apply multiplication gates U(a^2^i) to create the exponentiation
Quantum circuit with 35365 gates generated in 0.109 sec
Quantum circuit executed 1000 times in 1181.709 sec (got 892 different values)
Result x=20 (0.2%): (a^(x/2)-1, a^(x/2)+1): 19, 21     => factors:7,5
Found factors in 0.2 % of trials


In [ ]:
results = test_shor(N=55, list_for_a=[7], Nshots=5000, mincounts=2, version=1)
sum_probs = 0
for state,(x,p,q,prob) in results.items():     
    print(f"Result x={x} ({prob*100}%): (a^(x/2)-1, a^(x/2)+1): {x-1}, {x+1}     => factors:{p},{q}")
    sum_probs += prob*100
print(f"Found factors in {sum_probs:.1f} % of trials")

[7]
Number to factorize N=55; coprime a=7; total number of qubits: 26 (?)

Create quantum and classical registers
initialize qr_mul to |0...001> and equal superposition in qr_qft
Apply multiplication gates U(a^2^i) to create the exponentiation
Quantum circuit with 35365 gates generated in 0.157 sec


### 1c - (extra credit 30pts) Finally, choose a number 156 < N < 256, which is the product of 2 primes, and factorize N using the IonQ Forte-1 QPU.

In [14]:
# results = test_shor(N=187, list_for_a=[3], Nshots=100, mincounts=2, version=2)
# sum_probs = 0
# for state,(x,p,q,prob) in results.items():     
#     print(f"Result x={x} ({prob*100}%): (a^(x/2)-1, a^(x/2)+1): {x-1}, {x+1}     => factors:{p},{q}")
#     sum_probs += prob*100
# print(f"Found factors in {sum_probs:.1f} % of trials")